In [2]:
import csv
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, LSTM, Conv1D, Flatten
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import tensorflow as tf

# 1. Utworzenie zbioru treningowego i testowego

## 1.1 Otwarcie pliku

In [3]:
with open('data.csv', newline='') as f:
    reader = csv.reader(f)
    data_temp = list(reader)
data_temp.pop(0)
print(data_temp[:10])

[['0_3002', '0', '3002', '-0.014674097712154196', '0.20512820512820512', 'False', 'False'], ['0_3003', '0', '3003', '-0.1058639549288685', '0.1923076923076923', 'False', 'False'], ['0_3006', '0', '3006', '-0.1058639549288685', '0.1923076923076923', 'False', 'False'], ['0_3007', '0', '3007', '-0.1058639549288685', '0.1923076923076923', 'False', 'False'], ['0_3009', '0', '3009', '-0.014674097712154196', '0.20512820512820512', 'False', 'False'], ['0_3010', '0', '3010', '-0.014674097712154196', '0.20512820512820512', 'False', 'False'], ['0_3013', '0', '3013', '-0.014674097712154196', '0.20512820512820512', 'False', 'False'], ['0_3014', '0', '3014', '-0.014674097712154196', '0.20512820512820512', 'False', 'False'], ['0_3016', '0', '3016', '-0.014674097712154196', '0.20512820512820512', 'False', 'False'], ['0_3017', '0', '3017', '-0.014674097712154196', '0.20512820512820512', 'False', 'False']]


In [4]:
data = []
true = false = 0
for x in data_temp:
    if x[6] == 'True':
        true += 1
    else:
        false += 1
    data.append([x[0], float(x[3]), float(x[4]), x[5] == 'True', x[6] == 'True'])
print(data[0], true, false)

['0_3002', -0.014674097712154196, 0.20512820512820512, False, False] 21157 12170


## 1.2 Obliczenie rozmiaru

In [5]:
train_count = int(len(data) * 0.8)
test_count = len(data) - train_count
print(train_count + test_count, len(data))
assert train_count + test_count == len(data)

frame_size = 25

33327 33327


## 1.3 Utworzenie zbioru

Dla każdego Y dodawane jest do X frame_size wartości sensora przed

In [15]:
X_train_min_max = []
X_train_z_score = []
Y_train = []
X_test_min_max = []
X_test_z_score = []
Y_test = []

for i in range(9, train_count):
    ground_truth = data[i][4]
    Y_train.append(ground_truth)
    sensor_values_min_max = [data[j][2] for j in range(i - 9, i + 1)]
    sensor_values_z_score = [data[j][1] for j in range(i - 9, i + 1)]
    X_train_min_max.append(sensor_values_min_max)
    X_train_z_score.append(sensor_values_z_score)

for i in range(9 + train_count, len(data)):
    ground_truth = data[i][4]
    Y_test.append(ground_truth)
    sensor_values_min_max = [data[j][2] for j in range(i - 9, i + 1)]
    sensor_values_z_score = [data[j][1] for j in range(i - 9, i + 1)]
    X_test_min_max.append(sensor_values_min_max)
    X_test_z_score.append(sensor_values_z_score)

X_train_min_max = np.array(X_train_min_max)
X_train_z_score = np.array(X_train_z_score)
Y_train = np.array(Y_train)
X_test_min_max = np.array(X_test_min_max)
X_test_z_score = np.array(X_test_z_score)
Y_test = np.array(Y_test)

In [16]:
# y_history = {'true': 0, 'false': 0}
#
# for i, x in enumerate(data[:train_count]):
#     X_train_min_max.append(float(x[1]))
#     if x[3]:
#         y_history['true'] += 1
#     else:
#         y_history['false'] += 1
#
#     if i % frame_size == 0:
#         Y_train.append(y_history['true'] > y_history['false'])
#         y_history = {'true': 0, 'false': 0}
#
# y_history = {'true': 0, 'false': 0}
# print(len(data), len(data[train_count:test_count]), train_count, test_count)
# for i, x in enumerate(data[train_count: train_count + test_count]):
#     X_test_min_max.append(float(x[1]))
#     if x[3]:
#         y_history['true'] += 1
#     else:
#         y_history['false'] += 1
#
#     if i % frame_size == 0:
#         Y_test.append(y_history['true'] > y_history['false'])
#         y_history = {'true': 0, 'false': 0}
#
# print(len(X_test_min_max), len(Y_test))
#


# e = int(X_train.shape[0] / 25)
# X_train_min_max = X_train_min_max.reshape((int(train_count / frame_size), frame_size, 1))
# X_test_min_max = X_test_min_max.reshape((int(test_count / frame_size), frame_size, 1))
# print(X_train_min_max.shape)





## Definicja modelu

In [73]:
model = Sequential()
model.add(LSTM(32, input_shape=(10, 1), dropout=0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.summary()

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_15 (LSTM)              (None, 32)                4352      
                                                                 
 dense_49 (Dense)            (None, 128)               4224      
                                                                 
 dense_50 (Dense)            (None, 16)                2064      
                                                                 
 dense_51 (Dense)            (None, 1)                 17        
                                                                 
Total params: 10,657
Trainable params: 10,657
Non-trainable params: 0
_________________________________________________________________


In [74]:
model.fit(X_train_min_max, Y_train, validation_data=(X_test_min_max, Y_test), epochs=30)

Epoch 1/30
833/833 [==============================] - 5s 4ms/step - loss: 0.4748 - binary_accuracy: 0.7302 - precision_1: 0.8402 - recall: 0.7061 - val_loss: 0.7624 - val_binary_accuracy: 0.3850 - val_precision_1: 1.0000 - val_recall: 0.0614
Epoch 2/30
833/833 [==============================] - 3s 3ms/step - loss: 0.4404 - binary_accuracy: 0.7549 - precision_1: 0.9137 - recall: 0.6747 - val_loss: 0.7826 - val_binary_accuracy: 0.3861 - val_precision_1: 1.0000 - val_recall: 0.0630
Epoch 3/30
833/833 [==============================] - 3s 3ms/step - loss: 0.4267 - binary_accuracy: 0.7678 - precision_1: 0.9410 - recall: 0.6738 - val_loss: 0.7655 - val_binary_accuracy: 0.3924 - val_precision_1: 1.0000 - val_recall: 0.0727
Epoch 4/30
833/833 [==============================] - 3s 3ms/step - loss: 0.4198 - binary_accuracy: 0.7730 - precision_1: 0.9461 - recall: 0.6784 - val_loss: 0.7675 - val_binary_accuracy: 0.3849 - val_precision_1: 1.0000 - val_recall: 0.0612
Epoch 5/30
833/833 [============

In [75]:
model2 = Sequential()
model2.add(Conv1D(64, 3, input_shape=(10, 1)))
model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dense(16, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [76]:
model2.fit(X_train_min_max, Y_train, validation_data=(X_test_min_max, Y_test), epochs=30)

Epoch 1/30
833/833 [==============================] - 3s 2ms/step - loss: 0.3883 - binary_accuracy: 0.8016 - precision_2: 0.9194 - recall_1: 0.7509 - val_loss: 0.6873 - val_binary_accuracy: 0.4215 - val_precision_2: 1.0000 - val_recall_1: 0.1171
Epoch 2/30
833/833 [==============================] - 1s 1ms/step - loss: 0.3598 - binary_accuracy: 0.8168 - precision_2: 0.9617 - recall_1: 0.7387 - val_loss: 0.7058 - val_binary_accuracy: 0.4212 - val_precision_2: 1.0000 - val_recall_1: 0.1167
Epoch 3/30
833/833 [==============================] - 1s 1ms/step - loss: 0.3502 - binary_accuracy: 0.8216 - precision_2: 0.9588 - recall_1: 0.7491 - val_loss: 0.6831 - val_binary_accuracy: 0.4992 - val_precision_2: 1.0000 - val_recall_1: 0.2357
Epoch 4/30
833/833 [==============================] - 1s 1ms/step - loss: 0.3462 - binary_accuracy: 0.8294 - precision_2: 0.9505 - recall_1: 0.7693 - val_loss: 0.6374 - val_binary_accuracy: 0.5642 - val_precision_2: 0.6207 - val_recall_1: 0.8611
Epoch 5/30
833/8